# 预算分配优化 - 把钱花在刀刃上

> *"不是钱不够，是没花对地方"*

## 开场故事

想象你是一家电商公司的营销总监，手握 1000 万广告预算，面对 5 个投放渠道：

- **搜索广告**：精准但贵，单次点击 10 元
- **信息流**：量大便宜，单次点击 2 元
- **KOL 合作**：品效合一，但不确定性高
- **短视频**：年轻用户多，转化率一般
- **户外广告**：曝光量大，但难以追踪

你会怎么分配这 1000 万？平均分？全投 ROI 最高的？还是...

这就是我们今天要解决的问题：**预算分配优化**。

---

## 学习路线图

```
预算分配问题 → 边际 ROI → 约束优化 → 多渠道分配 → 稳健优化 → 实战案例
    ↓              ↓           ↓            ↓             ↓          ↓
  为什么优化    收益递减      数学建模     联合优化      应对不确定   业务落地
```

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.optimize import minimize, LinearConstraint, NonlinearConstraint
from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')

# 设置随机种子
np.random.seed(42)

# 颜色方案
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'warning': '#F2C94C',
    'danger': '#EB5757',
    'purple': '#9B51E0',
    'teal': '#56CCF2'
}

print("✓ 环境准备完成")

---

# Part 1: 预算分配问题

## 1.1 为什么需要优化？

### 常见的预算分配方式（都有问题！）

| 方式 | 描述 | 问题 |
|------|------|------|
| **平均分配** | 每个渠道分 200 万 | 忽略渠道效果差异 |
| **按历史占比** | 去年怎么分今年就怎么分 | 市场在变化 |
| **全投最优渠道** | ROI 最高的渠道全投 | 边际收益递减 |
| **拍脑袋决策** | 老板说这个渠道重要 | 缺乏数据支撑 |

### 优化的核心思想

**让每一块钱的边际收益相等**

就像给多个水桶加水，最优策略是：让所有水桶的**水面高度相同**（边际效用相等），而不是让某个桶最满。

## 1.2 边际收益递减原理

### 生活中的例子

- **吃火锅**：第一盘肥牛很香，第五盘就腻了
- **刷题**：前 100 道提升大，后 100 道收益递减
- **广告投放**：前 100 万触达精准用户，后 100 万重复曝光

### 数学表达

响应曲线（Response Curve）：

$$
R(x) = a \cdot \frac{x^\alpha}{c^\alpha + x^\alpha}
$$

其中：
- $R(x)$: 投入 $x$ 带来的收益
- $a$: 饱和收益（最大收益）
- $c$: 半饱和点（达到 50% 最大收益时的投入）
- $\alpha$: 形状参数（通常 0.5-2）

边际收益：

$$
R'(x) = \frac{dR}{dx} = a \cdot \alpha \cdot \frac{c^\alpha \cdot x^{\alpha-1}}{(c^\alpha + x^\alpha)^2}
$$

**关键性质**：$R'(x)$ 随 $x$ 递减（边际收益递减）

In [ ]:
# 可视化：边际收益递减

def response_curve(x, a, c, alpha):
    """响应曲线（Hill Equation）"""
    return a * (x**alpha) / (c**alpha + x**alpha)

def marginal_response(x, a, c, alpha):
    """边际响应（导数）"""
    return a * alpha * (c**alpha) * (x**(alpha-1)) / ((c**alpha + x**alpha)**2)

# 参数设置：3 个渠道
channels = {
    '搜索广告': {'a': 500, 'c': 150, 'alpha': 0.8},  # 高效但快饱和
    '信息流': {'a': 800, 'c': 300, 'alpha': 1.2},    # 潜力大但需要量
    '短视频': {'a': 600, 'c': 200, 'alpha': 1.0}     # 中等
}

x = np.linspace(0, 500, 200)

# 创建子图
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['响应曲线 R(x)', '边际响应 R\'(x)'],
    horizontal_spacing=0.12
)

colors_list = [COLORS['primary'], COLORS['success'], COLORS['warning']]

for i, (name, params) in enumerate(channels.items()):
    # 响应曲线
    y = response_curve(x, **params)
    fig.add_trace(
        go.Scatter(
            x=x, y=y, name=name,
            line=dict(color=colors_list[i], width=3),
            mode='lines'
        ),
        row=1, col=1
    )
    
    # 边际响应
    y_marginal = marginal_response(x, **params)
    fig.add_trace(
        go.Scatter(
            x=x, y=y_marginal, name=name,
            line=dict(color=colors_list[i], width=3, dash='dot'),
            mode='lines',
            showlegend=False
        ),
        row=1, col=2
    )

fig.update_xaxes(title_text="投入预算 (万元)", row=1, col=1)
fig.update_xaxes(title_text="投入预算 (万元)", row=1, col=2)
fig.update_yaxes(title_text="总收益 (万元)", row=1, col=1)
fig.update_yaxes(title_text="边际收益 (元/元)", row=1, col=2)

fig.update_layout(
    height=400,
    template='plotly_white',
    title_text="边际收益递减原理",
    hovermode='x unified'
)

fig.show()

print("\n💡 观察要点：")
print("1. 响应曲线：投入越多，收益增长越慢（S型曲线）")
print("2. 边际响应：投入越多，每增加1元的收益越低（递减曲线）")
print("3. 不同渠道有不同的饱和点和形状")

## 1.3 多目标权衡

实际业务中，预算分配往往需要考虑多个目标：

### 常见目标

1. **短期 ROI 最大化**：立即回本
2. **长期用户价值（LTV）**：品牌建设
3. **市场份额**：战略卡位
4. **风险分散**：不把鸡蛋放一个篮子

### 权衡方法

**加权目标函数**：

$$
\max_{x_1, \ldots, x_n} \quad w_1 \cdot ROI(x) + w_2 \cdot LTV(x) - w_3 \cdot Risk(x)
$$

其中 $w_1, w_2, w_3$ 是业务权重。

In [ ]:
# 可视化：多目标权衡

# 模拟数据：不同分配方案的表现
strategies = {
    '全投搜索': {'ROI': 3.5, 'LTV': 2.0, 'Risk': 8.0},
    '平均分配': {'ROI': 2.8, 'LTV': 3.5, 'Risk': 3.0},
    '优化分配': {'ROI': 3.2, 'LTV': 4.0, 'Risk': 2.5},
    '品牌导向': {'ROI': 2.0, 'LTV': 5.0, 'Risk': 4.0}
}

df = pd.DataFrame(strategies).T.reset_index()
df.columns = ['策略', 'ROI', 'LTV', '风险']

# 雷达图
fig = go.Figure()

categories = ['ROI', 'LTV', '风险（低更好）']

for i, row in df.iterrows():
    values = [row['ROI'], row['LTV'], 10 - row['风险']]  # 风险取反
    values += [values[0]]  # 闭合
    
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories + [categories[0]],
        name=row['策略'],
        fill='toself',
        line=dict(width=2)
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 10]
        )
    ),
    title="不同预算分配策略的多目标表现",
    template='plotly_white',
    height=500
)

fig.show()

print("\n💡 洞察：")
print("• 全投搜索：ROI高但风险大")
print("• 平均分配：保守稳健，但收益一般")
print("• 优化分配：在多个目标间取得平衡")
print("• 品牌导向：牺牲短期ROI，重视长期价值")

---

# Part 2: 边际 ROI 优化

## 2.1 边际 ROI 的定义

### ROI vs 边际 ROI

**平均 ROI**（容易误导决策）：
$$
ROI_{avg} = \frac{R(x)}{x}
$$

**边际 ROI**（优化的关键指标）：
$$
ROI_{marginal} = \frac{dR}{dx} = R'(x)
$$

### 生活中的比喻

- **平均成绩**：你考了 5 次试，平均 85 分
- **边际提升**：再学 1 小时，下次能提升几分？

决策时要看**边际提升**，不是平均分！

In [ ]:
# 可视化：平均 ROI vs 边际 ROI

def avg_roi(x, a, c, alpha):
    """平均 ROI"""
    return response_curve(x, a, c, alpha) / (x + 1e-10)

# 使用搜索广告的参数
params = channels['搜索广告']
x = np.linspace(1, 400, 200)

fig = go.Figure()

# 平均 ROI
fig.add_trace(go.Scatter(
    x=x,
    y=avg_roi(x, **params),
    name='平均 ROI',
    line=dict(color=COLORS['primary'], width=3)
))

# 边际 ROI
fig.add_trace(go.Scatter(
    x=x,
    y=marginal_response(x, **params),
    name='边际 ROI',
    line=dict(color=COLORS['danger'], width=3, dash='dash')
))

# 标注关键点
x_point = 200
fig.add_trace(go.Scatter(
    x=[x_point, x_point],
    y=[avg_roi(x_point, **params), marginal_response(x_point, **params)],
    mode='markers+text',
    marker=dict(size=12, color=['blue', 'red']),
    text=[f'平均: {avg_roi(x_point, **params):.2f}', 
          f'边际: {marginal_response(x_point, **params):.2f}'],
    textposition=['top center', 'bottom center'],
    showlegend=False
))

fig.update_layout(
    title="平均 ROI vs 边际 ROI（搜索广告）",
    xaxis_title="投入预算 (万元)",
    yaxis_title="ROI (元/元)",
    template='plotly_white',
    height=400,
    hovermode='x unified'
)

fig.show()

print("\n⚠️ 警示案例：")
print(f"假设已投入 {x_point} 万元：")
print(f"• 平均 ROI = {avg_roi(x_point, **params):.2f}，看起来不错")
print(f"• 但边际 ROI = {marginal_response(x_point, **params):.2f}，已经很低了")
print(f"• 再投入的钱应该分配给其他渠道！")

## 2.2 最优分配条件

### 拉格朗日乘数法的直觉

**问题**：预算 $B$，$n$ 个渠道，如何分配 $x_1, \ldots, x_n$？

**优化目标**：
$$
\max_{x_1, \ldots, x_n} \quad \sum_{i=1}^n R_i(x_i)
$$

**约束条件**：
$$
\sum_{i=1}^n x_i = B, \quad x_i \geq 0
$$

### 最优性条件（一阶条件）

$$
R_1'(x_1^*) = R_2'(x_2^*) = \cdots = R_n'(x_n^*) = \lambda
$$

**解读**：所有渠道的**边际 ROI 相等**，等于拉格朗日乘数 $\lambda$（影子价格）。

### 水桶模型

想象 $n$ 个形状不同的水桶（响应曲线不同）：
- 最优分配 = 让所有水桶的**水面高度相同**（边际 ROI 相等）
- 如果某个桶的水面更高（边际 ROI 更高），应该从其他桶转移水过来

In [ ]:
# TODO 1: 实现边际 ROI 相等的最优分配

def optimize_budget_marginal_equal(channels_params, total_budget):
    """
    使用数值优化找到边际 ROI 相等的预算分配
    
    参数:
        channels_params: dict, 每个渠道的参数 {'channel': {'a': ..., 'c': ..., 'alpha': ...}}
        total_budget: float, 总预算
    
    返回:
        allocation: dict, 最优分配方案
        metrics: dict, 性能指标
    """
    n_channels = len(channels_params)
    channel_names = list(channels_params.keys())
    
    # 目标函数：最大化总收益（负号转为最小化）
    def objective(x):
        total_response = 0
        for i, name in enumerate(channel_names):
            params = channels_params[name]
            total_response += response_curve(x[i], **params)
        return -total_response  # 负号：最大化转最小化
    
    # 约束条件
    constraints = [
        # 添加预算总和等于 total_budget 的约束
        LinearConstraint(np.ones(n_channels), total_budget, total_budget)
    ]
    
    # 边界：每个渠道预算 >= 0
    bounds = [(0, total_budget) for _ in range(n_channels)]
    
    # 初始猜测：平均分配
    x0 = np.ones(n_channels) * total_budget / n_channels
    
    # 使用 scipy.optimize.minimize 求解
    result = minimize(
        objective,
        x0,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    # 解析结果
    optimal_allocation = dict(zip(channel_names, result.x))
    
    # 计算边际 ROI（验证是否相等）
    marginal_rois = {}
    total_response = 0
    for name, budget in optimal_allocation.items():
        params = channels_params[name]
        marginal_rois[name] = marginal_response(budget, **params)
        total_response += response_curve(budget, **params)
    
    metrics = {
        'total_response': total_response,
        'avg_roi': total_response / total_budget,
        'marginal_rois': marginal_rois,
        'shadow_price': np.mean(list(marginal_rois.values()))
    }
    
    return optimal_allocation, metrics

# 测试
total_budget = 1000  # 1000 万
allocation, metrics = optimize_budget_marginal_equal(channels, total_budget)

print("\n🎯 最优预算分配：")
for channel, budget in allocation.items():
    print(f"  {channel}: {budget:.1f} 万元 ({budget/total_budget*100:.1f}%)")

print(f"\n📊 性能指标：")
print(f"  总收益: {metrics['total_response']:.1f} 万元")
print(f"  平均 ROI: {metrics['avg_roi']:.2f}")
print(f"  影子价格 λ: {metrics['shadow_price']:.3f}")

print(f"\n🔍 边际 ROI（验证是否相等）：")
for channel, mroi in metrics['marginal_rois'].items():
    print(f"  {channel}: {mroi:.3f}")

In [ ]:
# 可视化：最优分配 vs 其他策略

# 对比不同分配策略
strategies_comparison = {
    '平均分配': {ch: total_budget / len(channels) for ch in channels.keys()},
    '最优分配': allocation
}

# 计算每种策略的总收益
def calc_total_response(allocation, channels_params):
    total = 0
    for ch, budget in allocation.items():
        total += response_curve(budget, **channels_params[ch])
    return total

results = []
for strategy_name, alloc in strategies_comparison.items():
    total_resp = calc_total_response(alloc, channels)
    for ch, budget in alloc.items():
        results.append({
            '策略': strategy_name,
            '渠道': ch,
            '预算': budget,
            '收益': response_curve(budget, **channels[ch]),
            '边际ROI': marginal_response(budget, **channels[ch])
        })

df_results = pd.DataFrame(results)

# 分组柱状图
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['预算分配', '边际 ROI'],
    specs=[[{'type': 'bar'}, {'type': 'bar'}]]
)

for strategy in ['平均分配', '最优分配']:
    df_sub = df_results[df_results['策略'] == strategy]
    
    fig.add_trace(
        go.Bar(
            x=df_sub['渠道'],
            y=df_sub['预算'],
            name=strategy,
            text=df_sub['预算'].apply(lambda x: f'{x:.0f}'),
            textposition='auto'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=df_sub['渠道'],
            y=df_sub['边际ROI'],
            name=strategy,
            text=df_sub['边际ROI'].apply(lambda x: f'{x:.2f}'),
            textposition='auto',
            showlegend=False
        ),
        row=1, col=2
    )

fig.update_xaxes(title_text="渠道", row=1, col=1)
fig.update_xaxes(title_text="渠道", row=1, col=2)
fig.update_yaxes(title_text="预算 (万元)", row=1, col=1)
fig.update_yaxes(title_text="边际 ROI", row=1, col=2)

fig.update_layout(
    height=400,
    template='plotly_white',
    title_text="预算分配策略对比",
    barmode='group'
)

fig.show()

# 计算收益提升
avg_response = calc_total_response(strategies_comparison['平均分配'], channels)
opt_response = calc_total_response(strategies_comparison['最优分配'], channels)
improvement = (opt_response - avg_response) / avg_response * 100

print(f"\n💰 优化效果：")
print(f"  平均分配总收益: {avg_response:.1f} 万元")
print(f"  最优分配总收益: {opt_response:.1f} 万元")
print(f"  提升: {improvement:.1f}%")

---

# Part 3: 约束优化建模

## 3.1 实际业务约束

现实中的预算分配远比理论复杂，需要考虑各种业务约束：

### 常见约束类型

| 约束类型 | 数学表达 | 业务场景 |
|----------|----------|----------|
| **最小预算** | $x_i \geq x_i^{min}$ | 保持渠道活跃 |
| **最大预算** | $x_i \leq x_i^{max}$ | 供给限制 |
| **占比约束** | $x_i \leq \alpha \cdot B$ | 风险控制 |
| **分组约束** | $\sum_{i \in G} x_i \leq B_G$ | 品牌预算上限 |
| **依赖关系** | $x_i \geq \beta \cdot x_j$ | 协同效应 |

### 完整优化模型

$$
\begin{align}
\max_{x_1, \ldots, x_n} \quad & \sum_{i=1}^n R_i(x_i) \\
\text{s.t.} \quad & \sum_{i=1}^n x_i = B \\
& x_i^{min} \leq x_i \leq x_i^{max}, \quad \forall i \\
& x_i \leq \alpha_i \cdot B, \quad \forall i \\
& \text{其他业务约束}
\end{align}
$$

In [ ]:
# TODO 2: 实现带约束的预算优化

def optimize_with_constraints(channels_params, total_budget, constraints_config):
    """
    带业务约束的预算优化
    
    参数:
        channels_params: dict, 渠道参数
        total_budget: float, 总预算
        constraints_config: dict, 约束配置
            {
                'min_budget': {channel: min_value},  # 最小预算
                'max_budget': {channel: max_value},  # 最大预算
                'max_share': {channel: max_ratio}    # 最大占比
            }
    """
    n_channels = len(channels_params)
    channel_names = list(channels_params.keys())
    
    # 目标函数
    def objective(x):
        total_response = 0
        for i, name in enumerate(channel_names):
            params = channels_params[name]
            total_response += response_curve(x[i], **params)
        return -total_response
    
    # 构建约束条件列表
    constraints = []
    
    # 1. 预算总和约束
    constraints.append(
        LinearConstraint(np.ones(n_channels), total_budget, total_budget)
    )
    
    # 边界约束
    bounds = []
    for i, name in enumerate(channel_names):
        min_b = constraints_config.get('min_budget', {}).get(name, 0)
        max_b = constraints_config.get('max_budget', {}).get(name, total_budget)
        
        # 考虑占比约束
        if name in constraints_config.get('max_share', {}):
            max_share = constraints_config['max_share'][name]
            max_b = min(max_b, total_budget * max_share)
        
        bounds.append((min_b, max_b))
    
    # 初始猜测
    x0 = np.ones(n_channels) * total_budget / n_channels
    
    # 优化求解
    result = minimize(
        objective,
        x0,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints,
        options={'maxiter': 1000}
    )
    
    if not result.success:
        print(f"⚠️ 优化警告: {result.message}")
    
    optimal_allocation = dict(zip(channel_names, result.x))
    total_response = -result.fun
    
    return optimal_allocation, total_response

# 定义业务约束
constraints_config = {
    'min_budget': {
        '搜索广告': 50,   # 至少投 50 万保持活跃
        '信息流': 100,    # 信息流需要量
        '短视频': 30
    },
    'max_budget': {
        '搜索广告': 400,  # 搜索广告供给有限
    },
    'max_share': {
        '搜索广告': 0.5,  # 单渠道不超过 50%（风险控制）
    }
}

# 优化
allocation_constrained, response_constrained = optimize_with_constraints(
    channels, total_budget, constraints_config
)

print("\n🎯 带约束的最优分配：")
for channel, budget in allocation_constrained.items():
    print(f"  {channel}: {budget:.1f} 万元 ({budget/total_budget*100:.1f}%)")
    
    # 检查约束
    if channel in constraints_config['min_budget']:
        min_b = constraints_config['min_budget'][channel]
        if budget <= min_b + 1:
            print(f"    ⚠️ 触及最小预算约束 {min_b}")
    
    if channel in constraints_config['max_budget']:
        max_b = constraints_config['max_budget'][channel]
        if budget >= max_b - 1:
            print(f"    ⚠️ 触及最大预算约束 {max_b}")

print(f"\n📊 总收益: {response_constrained:.1f} 万元")

# 对比无约束优化
response_unconstrained = calc_total_response(allocation, channels)
loss = (response_unconstrained - response_constrained) / response_unconstrained * 100
print(f"\n💡 约束导致的收益损失: {loss:.1f}%")

## 3.2 拉格朗日方法的经济学解释

### 拉格朗日函数

$$
\mathcal{L}(x, \lambda) = \sum_{i=1}^n R_i(x_i) - \lambda \left( \sum_{i=1}^n x_i - B \right)
$$

### 影子价格（Shadow Price）

拉格朗日乘数 $\lambda$ 表示：
- **再增加 1 元预算，总收益会增加多少**
- 在最优点，$\lambda$ = 所有渠道的边际 ROI

### 实际应用

如果老板问："再给你 100 万，能提升多少收益？"

答案就是：$\Delta R \approx \lambda \cdot 100$（线性近似）

In [ ]:
# 可视化：影子价格分析

# 测试不同预算下的最优收益和影子价格
budgets = np.linspace(500, 2000, 20)
responses = []
shadow_prices = []

for B in budgets:
    alloc, metrics = optimize_budget_marginal_equal(channels, B)
    responses.append(metrics['total_response'])
    shadow_prices.append(metrics['shadow_price'])

# 计算数值导数（验证影子价格）
numerical_derivatives = np.gradient(responses, budgets)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['总收益曲线', '影子价格（边际收益）']
)

# 总收益
fig.add_trace(
    go.Scatter(
        x=budgets,
        y=responses,
        mode='lines+markers',
        name='总收益',
        line=dict(color=COLORS['primary'], width=3)
    ),
    row=1, col=1
)

# 影子价格 vs 数值导数
fig.add_trace(
    go.Scatter(
        x=budgets,
        y=shadow_prices,
        mode='lines+markers',
        name='影子价格 λ',
        line=dict(color=COLORS['success'], width=3)
    ),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(
        x=budgets,
        y=numerical_derivatives,
        mode='lines',
        name='数值导数 dR/dB',
        line=dict(color=COLORS['danger'], width=2, dash='dash')
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="总预算 (万元)", row=1, col=1)
fig.update_xaxes(title_text="总预算 (万元)", row=1, col=2)
fig.update_yaxes(title_text="总收益 (万元)", row=1, col=1)
fig.update_yaxes(title_text="边际收益 (元/元)", row=1, col=2)

fig.update_layout(
    height=400,
    template='plotly_white',
    title_text="影子价格分析",
    hovermode='x unified'
)

fig.show()

print("\n💡 洞察：")
print("1. 影子价格 λ 与数值导数 dR/dB 基本一致（验证了理论）")
print("2. 随着预算增加，影子价格递减（整体边际收益递减）")
print(f"3. 当前预算 {total_budget} 万时，λ ≈ {metrics['shadow_price']:.3f}")
print(f"   → 再增加 100 万预算，预期增加收益约 {metrics['shadow_price']*100:.1f} 万")

---

# Part 4: 多渠道分配实战

## 4.1 渠道间的交互效应

现实中，渠道之间不是独立的：

### 协同效应（Synergy）

某些渠道组合会产生 1+1>2 的效果：

$$
R(x_1, x_2) = R_1(x_1) + R_2(x_2) + \gamma \cdot \sqrt{x_1 \cdot x_2}
$$

**例子**：
- 品牌广告（户外）+ 效果广告（搜索）：品牌提升搜索转化率
- KOL 种草 + 信息流收割：上下游配合

### 替代效应（Substitution）

某些渠道会抢夺同一批用户：

$$
R(x_1, x_2) = R_1(x_1) + R_2(x_2) - \delta \cdot \min(x_1, x_2)
$$

**例子**：
- 搜索广告 vs 信息流：同时触达同一用户（浪费）

In [ ]:
# TODO 3: 实现考虑交互效应的预算优化

def optimize_with_interaction(channels_params, total_budget, interaction_matrix):
    """
    考虑渠道交互效应的预算优化
    
    参数:
        channels_params: dict, 渠道参数
        total_budget: float, 总预算
        interaction_matrix: np.ndarray, 交互矩阵
            interaction_matrix[i, j] > 0: 协同效应
            interaction_matrix[i, j] < 0: 替代效应
    """
    n_channels = len(channels_params)
    channel_names = list(channels_params.keys())
    
    # 定义包含交互项的目标函数
    def objective(x):
        # 基础收益
        total_response = 0
        for i, name in enumerate(channel_names):
            params = channels_params[name]
            total_response += response_curve(x[i], **params)
        
        # 交互效应
        for i in range(n_channels):
            for j in range(i+1, n_channels):
                gamma = interaction_matrix[i, j]
                if gamma != 0:
                    # 使用几何平均建模交互
                    interaction_term = gamma * np.sqrt(x[i] * x[j] + 1e-10)
                    total_response += interaction_term
        
        return -total_response
    
    # 约束和边界
    constraints = [
        LinearConstraint(np.ones(n_channels), total_budget, total_budget)
    ]
    bounds = [(0, total_budget) for _ in range(n_channels)]
    x0 = np.ones(n_channels) * total_budget / n_channels
    
    # 优化
    result = minimize(
        objective,
        x0,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    optimal_allocation = dict(zip(channel_names, result.x))
    total_response = -result.fun
    
    return optimal_allocation, total_response

# 定义交互矩阵（对称矩阵）
# 0: 搜索广告, 1: 信息流, 2: 短视频
interaction_matrix = np.array([
    [0,    -0.2,  0.3],   # 搜索：与信息流替代，与短视频协同
    [-0.2,  0,    0.1],   # 信息流：与短视频弱协同
    [0.3,   0.1,  0]      # 短视频
])

print("📊 渠道交互矩阵：")
print("   搜索  信息流  短视频")
for i, name in enumerate(channels.keys()):
    print(f"{name:6s} {interaction_matrix[i]}")
print("\n说明: >0 协同, <0 替代")

# 优化（有交互 vs 无交互）
allocation_no_interact, response_no_interact = optimize_budget_marginal_equal(channels, total_budget)
allocation_with_interact, response_with_interact = optimize_with_interaction(
    channels, total_budget, interaction_matrix
)

print("\n🎯 分配对比：")
print(f"{'渠道':<10} {'无交互':<12} {'有交互':<12} {'变化'}")
print("-" * 50)
for ch in channels.keys():
    no_int = allocation_no_interact[ch]
    with_int = allocation_with_interact[ch]
    change = with_int - no_int
    print(f"{ch:<10} {no_int:>8.1f} 万   {with_int:>8.1f} 万   {change:+7.1f}")

print(f"\n📊 总收益对比：")
print(f"  无交互: {response_no_interact:.1f} 万元")
print(f"  有交互: {response_with_interact:.1f} 万元")
print(f"  提升: {(response_with_interact - response_no_interact):.1f} 万元")

## 4.2 实际操作流程

### 预算优化的完整流程

```mermaid
graph TD
    A[数据收集] --> B[响应曲线建模]
    B --> C[定义约束条件]
    C --> D[优化求解]
    D --> E[敏感性分析]
    E --> F[业务评审]
    F --> G{通过?}
    G -->|是| H[执行分配]
    G -->|否| C
    H --> I[效果监控]
    I --> J[参数更新]
    J --> B
```

### 关键步骤详解

1. **数据收集**（2-4周）
   - 历史投放数据
   - 不同预算水平的效果
   - A/B 测试数据

2. **响应曲线建模**（1周）
   - 拟合 Hill 方程或 Logistic 曲线
   - 估计参数（a, c, α）
   - 模型验证

3. **定义约束**（与业务讨论）
   - 最小预算（保持活跃）
   - 最大预算（供给限制）
   - 风险控制（占比上限）

4. **优化求解**（数小时）
   - 使用 SLSQP / IPOPT 求解器
   - 检查收敛性

5. **敏感性分析**（1-2天）
   - 参数扰动
   - What-if 分析

6. **业务评审**（会议）
   - 向管理层汇报
   - 调整约束

7. **效果监控**（持续）
   - 实际 vs 预测
   - 模型校准

In [ ]:
# 实战工具：预算优化器类

class BudgetOptimizer:
    """
    预算优化器
    
    封装完整的优化流程
    """
    
    def __init__(self, channels_params):
        self.channels = channels_params
        self.channel_names = list(channels_params.keys())
        self.n_channels = len(channels_params)
        self.history = []
    
    def fit_response_curve(self, data):
        """从历史数据拟合响应曲线"""
        # 简化版：这里假设参数已给定
        # 实际应该用非线性回归拟合
        pass
    
    def optimize(self, total_budget, constraints=None, interaction_matrix=None):
        """执行优化"""
        n = self.n_channels
        
        def objective(x):
            total = 0
            for i, name in enumerate(self.channel_names):
                total += response_curve(x[i], **self.channels[name])
            
            # 交互项
            if interaction_matrix is not None:
                for i in range(n):
                    for j in range(i+1, n):
                        gamma = interaction_matrix[i, j]
                        if gamma != 0:
                            total += gamma * np.sqrt(x[i] * x[j] + 1e-10)
            
            return -total
        
        # 约束
        cons = [LinearConstraint(np.ones(n), total_budget, total_budget)]
        
        # 边界
        bounds = [(0, total_budget) for _ in range(n)]
        if constraints and 'min_budget' in constraints:
            for i, name in enumerate(self.channel_names):
                if name in constraints['min_budget']:
                    bounds[i] = (constraints['min_budget'][name], bounds[i][1])
        
        x0 = np.ones(n) * total_budget / n
        result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)
        
        allocation = dict(zip(self.channel_names, result.x))
        total_response = -result.fun
        
        # 记录历史
        self.history.append({
            'budget': total_budget,
            'allocation': allocation,
            'response': total_response
        })
        
        return allocation, total_response
    
    def sensitivity_analysis(self, base_budget, budget_range):
        """敏感性分析：不同预算下的最优分配"""
        results = []
        for B in budget_range:
            alloc, resp = self.optimize(B)
            results.append({'budget': B, 'response': resp, 'allocation': alloc})
        return pd.DataFrame(results)
    
    def what_if(self, scenario_name, **kwargs):
        """What-if 分析"""
        print(f"\n📊 Scenario: {scenario_name}")
        allocation, response = self.optimize(**kwargs)
        
        for ch, budget in allocation.items():
            print(f"  {ch}: {budget:.1f} 万元")
        print(f"  总收益: {response:.1f} 万元")
        
        return allocation, response

# 使用示例
optimizer = BudgetOptimizer(channels)

# Scenario 1: 基准情况
optimizer.what_if(
    "基准情况",
    total_budget=1000
)

# Scenario 2: 预算削减 20%
optimizer.what_if(
    "预算削减 20%",
    total_budget=800
)

# Scenario 3: 预算增加，但搜索广告受限
optimizer.what_if(
    "预算增加，搜索受限",
    total_budget=1200,
    constraints={'min_budget': {'搜索广告': 50}, 'max_budget': {'搜索广告': 300}}
)

---

# Part 5: 不确定性与稳健优化

## 5.1 参数估计的不确定性

响应曲线的参数（a, c, α）是从历史数据**估计**的，存在不确定性：

$$
\hat{\theta} = (\hat{a}, \hat{c}, \hat{\alpha}) \sim \mathcal{N}(\theta^*, \Sigma)
$$

### 不确定性的来源

1. **数据噪声**：观测误差
2. **样本量有限**：估计方差
3. **模型误设**：响应曲线可能不准确
4. **外部冲击**：市场变化、竞争对手

### 稳健优化的目标

找到在参数不确定性下**仍然表现良好**的分配方案。

## 5.2 稳健优化方法

### 方法 1: 期望最大化

$$
\max_{x} \quad \mathbb{E}_{\theta} \left[ \sum_i R_i(x_i; \theta) \right]
$$

平均来看最优（风险中性）

### 方法 2: 最坏情况优化（Worst-Case）

$$
\max_{x} \min_{\theta \in \Theta} \quad \sum_i R_i(x_i; \theta)
$$

保守策略，保证最坏情况不太差（风险厌恶）

### 方法 3: CVaR 优化（Conditional Value at Risk）

$$
\max_{x} \quad \text{CVaR}_{\alpha}\left[ \sum_i R_i(x_i; \theta) \right]
$$

优化最差的 α% 情况下的平均收益（平衡期望和风险）

In [ ]:
# 稳健优化示例：蒙特卡洛方法

def robust_optimization_mc(channels_params, total_budget, param_uncertainty, n_scenarios=1000):
    """
    蒙特卡洛稳健优化
    
    参数:
        channels_params: dict, 渠道参数的均值
        total_budget: float, 总预算
        param_uncertainty: dict, 参数的标准差
            {'channel': {'a_std': ..., 'c_std': ..., 'alpha_std': ...}}
        n_scenarios: int, 场景数量
    """
    n_channels = len(channels_params)
    channel_names = list(channels_params.keys())
    
    # 生成参数场景
    scenarios = []
    for _ in range(n_scenarios):
        scenario = {}
        for name in channel_names:
            base = channels_params[name]
            unc = param_uncertainty.get(name, {'a_std': 0, 'c_std': 0, 'alpha_std': 0})
            
            # 采样参数（确保 > 0）
            scenario[name] = {
                'a': max(0.1, np.random.normal(base['a'], unc.get('a_std', 0))),
                'c': max(1, np.random.normal(base['c'], unc.get('c_std', 0))),
                'alpha': max(0.1, np.random.normal(base['alpha'], unc.get('alpha_std', 0)))
            }
        scenarios.append(scenario)
    
    # 优化：期望最大化
    def objective_expected(x):
        expected_response = 0
        for scenario in scenarios:
            scenario_response = 0
            for i, name in enumerate(channel_names):
                scenario_response += response_curve(x[i], **scenario[name])
            expected_response += scenario_response
        expected_response /= n_scenarios
        return -expected_response
    
    # 求解
    constraints = [LinearConstraint(np.ones(n_channels), total_budget, total_budget)]
    bounds = [(0, total_budget) for _ in range(n_channels)]
    x0 = np.ones(n_channels) * total_budget / n_channels
    
    result = minimize(
        objective_expected,
        x0,
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    allocation = dict(zip(channel_names, result.x))
    
    # 评估该分配在所有场景下的表现
    performances = []
    for scenario in scenarios:
        perf = sum(response_curve(allocation[name], **scenario[name]) for name in channel_names)
        performances.append(perf)
    
    performances = np.array(performances)
    
    stats = {
        'mean': np.mean(performances),
        'std': np.std(performances),
        'min': np.min(performances),
        'percentile_5': np.percentile(performances, 5),
        'percentile_95': np.percentile(performances, 95),
        'max': np.max(performances)
    }
    
    return allocation, stats, performances

# 定义参数不确定性（标准差）
param_uncertainty = {
    '搜索广告': {'a_std': 50, 'c_std': 20, 'alpha_std': 0.1},
    '信息流': {'a_std': 80, 'c_std': 40, 'alpha_std': 0.15},
    '短视频': {'a_std': 60, 'c_std': 30, 'alpha_std': 0.12}
}

# 稳健优化
allocation_robust, stats_robust, performances = robust_optimization_mc(
    channels, total_budget, param_uncertainty, n_scenarios=500
)

print("\n🎯 稳健优化分配：")
for ch, budget in allocation_robust.items():
    print(f"  {ch}: {budget:.1f} 万元")

print(f"\n📊 性能分布（500 个场景）：")
print(f"  期望收益: {stats_robust['mean']:.1f} 万元")
print(f"  标准差: {stats_robust['std']:.1f} 万元")
print(f"  最差情况: {stats_robust['min']:.1f} 万元")
print(f"  5% 分位数: {stats_robust['percentile_5']:.1f} 万元")
print(f"  95% 分位数: {stats_robust['percentile_95']:.1f} 万元")
print(f"  最好情况: {stats_robust['max']:.1f} 万元")

In [ ]:
# 可视化：性能分布

fig = go.Figure()

# 直方图
fig.add_trace(go.Histogram(
    x=performances,
    nbinsx=30,
    name='收益分布',
    marker=dict(color=COLORS['primary'], opacity=0.7)
))

# 标注关键点
fig.add_vline(
    x=stats_robust['mean'],
    line_dash="dash",
    line_color="green",
    annotation_text=f"期望: {stats_robust['mean']:.0f}"
)

fig.add_vline(
    x=stats_robust['percentile_5'],
    line_dash="dot",
    line_color="red",
    annotation_text=f"5%: {stats_robust['percentile_5']:.0f}"
)

fig.add_vline(
    x=stats_robust['percentile_95'],
    line_dash="dot",
    line_color="red",
    annotation_text=f"95%: {stats_robust['percentile_95']:.0f}"
)

fig.update_layout(
    title="稳健优化：收益分布（500 个参数场景）",
    xaxis_title="总收益 (万元)",
    yaxis_title="频数",
    template='plotly_white',
    height=400
)

fig.show()

print("\n💡 解读：")
print("1. 收益分布近似正态（参数不确定性传导）")
print("2. 90% 置信区间：[{:.0f}, {:.0f}]".format(
    stats_robust['percentile_5'], stats_robust['percentile_95']
))
print("3. 即使在最差的 5% 情况下，仍能获得 {:.0f} 万收益".format(
    stats_robust['percentile_5']
))

## 5.3 敏感性分析

### Tornado 图（龙卷风图）

展示各参数变化对最优分配的影响：
- 横轴：参数变化范围（如 ±20%）
- 纵轴：对某渠道预算的影响
- 条形长度：敏感性大小

In [ ]:
# Tornado 图：参数敏感性分析

def sensitivity_tornado(channels_params, total_budget, param_name, param_range):
    """
    单参数敏感性分析
    
    参数:
        param_name: str, 如 '搜索广告.a'
        param_range: tuple, 如 (0.8, 1.2) 表示 ±20%
    """
    channel_name, param = param_name.split('.')
    base_value = channels_params[channel_name][param]
    
    results = []
    for factor in np.linspace(param_range[0], param_range[1], 20):
        # 修改参数
        modified_params = {k: v.copy() for k, v in channels_params.items()}
        modified_params[channel_name][param] = base_value * factor
        
        # 重新优化
        alloc, _ = optimize_budget_marginal_equal(modified_params, total_budget)
        
        results.append({
            'factor': factor,
            'param_value': base_value * factor,
            **{f'{ch}_budget': alloc[ch] for ch in channels_params.keys()}
        })
    
    return pd.DataFrame(results)

# 分析多个参数
params_to_test = [
    ('搜索广告.a', (0.8, 1.2)),
    ('信息流.c', (0.8, 1.2)),
    ('短视频.alpha', (0.8, 1.2))
]

fig = go.Figure()

for param_name, param_range in params_to_test:
    df_sens = sensitivity_tornado(channels, total_budget, param_name, param_range)
    
    # 绘制每个渠道的预算变化
    for ch in channels.keys():
        fig.add_trace(go.Scatter(
            x=df_sens['factor'],
            y=df_sens[f'{ch}_budget'],
            mode='lines',
            name=f"{param_name} → {ch}",
            line=dict(width=2)
        ))

fig.update_layout(
    title="参数敏感性分析：参数变化 → 预算分配变化",
    xaxis_title="参数变化倍数（1.0 = 基准）",
    yaxis_title="渠道预算 (万元)",
    template='plotly_white',
    height=500,
    hovermode='x unified'
)

fig.show()

print("\n💡 观察要点：")
print("1. 斜率陡的曲线：对该参数敏感")
print("2. 斜率平的曲线：对该参数不敏感（稳健）")
print("3. 交叉的曲线：参数变化导致渠道间预算重新分配")

---

# Part 6: 业务案例

## 案例 1: 电商双十一广告预算分配

### 背景

某电商平台双十一大促，准备 5000 万广告预算，投放 6 个渠道：

| 渠道 | 历史 ROI | 特点 |
|------|----------|------|
| 搜索广告 | 3.5 | 精准但贵 |
| 信息流 | 2.8 | 量大价优 |
| 短视频 | 2.2 | 年轻用户 |
| KOL 合作 | 3.0 | 不确定性高 |
| 直播带货 | 4.2 | 爆发力强 |
| 户外广告 | 1.5 | 品牌曝光 |

### 约束条件

1. 每个渠道至少投 200 万（保持活跃）
2. 直播带货最多投 1500 万（主播档期有限）
3. 线上渠道（前5个）至少占 80%（效果可追踪）
4. KOL 和直播有协同效应（系数 0.5）

In [ ]:
# 案例 1: 电商双十一

# 定义 6 个渠道（根据历史数据拟合的响应曲线）
channels_1111 = {
    '搜索广告': {'a': 1500, 'c': 400, 'alpha': 0.9},
    '信息流': {'a': 2500, 'c': 800, 'alpha': 1.1},
    '短视频': {'a': 2000, 'c': 700, 'alpha': 1.0},
    'KOL合作': {'a': 1800, 'c': 500, 'alpha': 0.8},
    '直播带货': {'a': 2200, 'c': 600, 'alpha': 1.2},
    '户外广告': {'a': 1000, 'c': 900, 'alpha': 0.7}
}

total_budget_1111 = 5000

# 交互矩阵（KOL 和直播协同）
interaction_1111 = np.zeros((6, 6))
interaction_1111[3, 4] = 0.5  # KOL (3) 和直播 (4) 协同
interaction_1111[4, 3] = 0.5

# 定义约束
def optimize_case1():
    n = 6
    channel_names = list(channels_1111.keys())
    
    def objective(x):
        total = 0
        for i, name in enumerate(channel_names):
            total += response_curve(x[i], **channels_1111[name])
        
        # 协同效应
        total += 0.5 * np.sqrt(x[3] * x[4] + 1e-10)  # KOL * 直播
        
        return -total
    
    # 约束
    constraints = [
        # 预算总和
        LinearConstraint(np.ones(n), total_budget_1111, total_budget_1111),
        # 线上渠道至少 80%
        LinearConstraint(
            [1, 1, 1, 1, 1, 0],  # 前 5 个是线上
            total_budget_1111 * 0.8,
            total_budget_1111
        )
    ]
    
    # 边界
    bounds = [
        (200, total_budget_1111),  # 搜索
        (200, total_budget_1111),  # 信息流
        (200, total_budget_1111),  # 短视频
        (200, total_budget_1111),  # KOL
        (200, 1500),               # 直播（上限 1500）
        (200, total_budget_1111)   # 户外
    ]
    
    x0 = np.ones(n) * total_budget_1111 / n
    result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)
    
    allocation = dict(zip(channel_names, result.x))
    total_response = -result.fun
    
    return allocation, total_response

allocation_1111, response_1111 = optimize_case1()

print("\n🎯 双十一最优预算分配（5000 万总预算）：\n")
print(f"{'渠道':<10} {'预算(万)':<12} {'占比':<10} {'预期收益(万)'}")
print("-" * 50)

for ch, budget in allocation_1111.items():
    revenue = response_curve(budget, **channels_1111[ch])
    print(f"{ch:<10} {budget:>8.0f}      {budget/total_budget_1111*100:>5.1f}%     {revenue:>8.0f}")

print("-" * 50)
print(f"{'总计':<10} {sum(allocation_1111.values()):>8.0f}      100.0%     {response_1111:>8.0f}")

# 计算整体 ROI
overall_roi = response_1111 / total_budget_1111
print(f"\n📊 整体 ROI: {overall_roi:.2f}")

# 检查约束
online_budget = sum(allocation_1111[ch] for ch in list(channels_1111.keys())[:5])
print(f"\n✓ 线上渠道占比: {online_budget/total_budget_1111*100:.1f}% (要求 ≥80%)")
print(f"✓ 直播预算: {allocation_1111['直播带货']:.0f} 万 (上限 1500 万)")

In [ ]:
# 可视化：预算分配饼图 + 收益对比

fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'bar'}]],
    subplot_titles=['预算分配', '各渠道收益贡献']
)

# 饼图
fig.add_trace(
    go.Pie(
        labels=list(allocation_1111.keys()),
        values=list(allocation_1111.values()),
        textinfo='label+percent',
        textposition='inside',
        hovertemplate='%{label}<br>%{value:.0f} 万元<br>%{percent}'
    ),
    row=1, col=1
)

# 柱状图
revenues = [response_curve(allocation_1111[ch], **channels_1111[ch]) for ch in channels_1111.keys()]
fig.add_trace(
    go.Bar(
        x=list(channels_1111.keys()),
        y=revenues,
        text=[f'{r:.0f}' for r in revenues],
        textposition='auto',
        marker=dict(color=COLORS['success'])
    ),
    row=1, col=2
)

fig.update_xaxes(tickangle=45, row=1, col=2)
fig.update_yaxes(title_text="收益 (万元)", row=1, col=2)

fig.update_layout(
    title_text="双十一预算分配方案",
    template='plotly_white',
    height=400,
    showlegend=False
)

fig.show()

## 案例 2: 优惠券类型预算分配

### 背景

某 O2O 平台发放优惠券，有 4 种类型：

| 券类型 | 成本 | 核销率 | 客单价提升 | ROI |
|--------|------|--------|------------|-----|
| 满减券 | 5元 | 60% | +15元 | 1.8 |
| 折扣券 | 8元 | 70% | +20元 | 1.75 |
| 新人券 | 15元 | 80% | +50元 | 2.67 |
| 会员券 | 3元 | 50% | +10元 | 1.67 |

总预算 2000 万，如何分配？

### 特殊考虑

- 新人券有增长战略价值（LTV 更高）
- 会员券维护老客户（留存重要）
- 满减和折扣有部分用户重叠（替代效应）

In [ ]:
# 案例 2: 优惠券分配

# 响应曲线（基于历史核销数据拟合）
coupon_types = {
    '满减券': {'a': 800, 'c': 350, 'alpha': 1.0},
    '折扣券': {'a': 750, 'c': 300, 'alpha': 0.95},
    '新人券': {'a': 1200, 'c': 400, 'alpha': 1.1},
    '会员券': {'a': 600, 'c': 250, 'alpha': 0.85}
}

total_budget_coupon = 2000

# 交互效应（满减和折扣有替代）
interaction_coupon = np.array([
    [0,    -0.15,  0,     0.1],   # 满减：与折扣替代，与会员协同
    [-0.15, 0,     0,     0],     # 折扣
    [0,     0,     0,     0.2],   # 新人：与会员协同（新老结合）
    [0.1,   0,     0.2,   0]      # 会员
])

# 优化（考虑多目标：短期 ROI + 长期价值）
def optimize_coupon_with_ltv(ltv_weights):
    """
    ltv_weights: dict, 每种券的长期价值权重
    """
    n = 4
    coupon_names = list(coupon_types.keys())
    
    def objective(x):
        # 短期收益
        short_term = 0
        for i, name in enumerate(coupon_names):
            short_term += response_curve(x[i], **coupon_types[name])
        
        # 交互效应
        for i in range(n):
            for j in range(i+1, n):
                gamma = interaction_coupon[i, j]
                if gamma != 0:
                    short_term += gamma * np.sqrt(x[i] * x[j] + 1e-10)
        
        # 长期价值（加权）
        long_term = sum(ltv_weights[name] * x[i] for i, name in enumerate(coupon_names))
        
        # 综合目标（70% 短期 + 30% 长期）
        total = 0.7 * short_term + 0.3 * long_term
        
        return -total
    
    constraints = [
        LinearConstraint(np.ones(n), total_budget_coupon, total_budget_coupon)
    ]
    bounds = [(100, total_budget_coupon) for _ in range(n)]  # 每种至少 100 万
    x0 = np.ones(n) * total_budget_coupon / n
    
    result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)
    
    allocation = dict(zip(coupon_names, result.x))
    total_value = -result.fun
    
    return allocation, total_value

# LTV 权重（归一化后的相对价值）
ltv_weights = {
    '满减券': 0.3,
    '折扣券': 0.25,
    '新人券': 0.8,   # 新客户 LTV 高
    '会员券': 0.5    # 老客户留存价值
}

allocation_coupon, value_coupon = optimize_coupon_with_ltv(ltv_weights)

print("\n🎯 优惠券预算分配（2000 万总预算）：\n")
print(f"{'券类型':<10} {'预算(万)':<12} {'占比':<10} {'预期短期ROI'}")
print("-" * 50)

total_short_term_revenue = 0
for name, budget in allocation_coupon.items():
    revenue = response_curve(budget, **coupon_types[name])
    total_short_term_revenue += revenue
    roi = revenue / budget
    print(f"{name:<10} {budget:>8.0f}      {budget/total_budget_coupon*100:>5.1f}%     {roi:>6.2f}")

print("-" * 50)
overall_short_roi = total_short_term_revenue / total_budget_coupon
print(f"\n📊 整体短期 ROI: {overall_short_roi:.2f}")
print(f"📊 综合价值（含 LTV）: {value_coupon:.0f}")

print("\n💡 解读：")
print(f"• 新人券占比 {allocation_coupon['新人券']/total_budget_coupon*100:.1f}%，战略重点")
print(f"• 满减券虽然短期ROI不是最高，但与会员券协同")
print(f"• 考虑 LTV 后，更倾向于长期价值高的券种")

---

# Part 7: 练习与思考题

## 练习 1: 实现 CVaR 优化

在前面的稳健优化中，我们使用了期望最大化。现在请实现 CVaR（Conditional Value at Risk）优化：

$$
\max_{x} \quad \text{CVaR}_{0.1}[R(x)] = \mathbb{E}[R(x) \mid R(x) \leq F^{-1}(0.1)]
$$

即：优化最差的 10% 情况下的平均收益。

**提示**：
1. 生成多个参数场景
2. 对每个分配方案，计算所有场景下的收益
3. 取最差的 10% 场景，计算平均值作为目标

In [ ]:
# TODO: 练习 1 - CVaR 优化

def optimize_cvar(channels_params, total_budget, param_uncertainty, alpha=0.1, n_scenarios=500):
    """
    CVaR 优化
    
    参数:
        alpha: float, 风险水平（如 0.1 表示最差的 10%）
    
    返回:
        allocation: dict, 最优分配
        cvar: float, CVaR 值
    """
    # TODO: 实现 CVaR 优化
    # 步骤:
    # 1. 生成参数场景
    # 2. 定义目标函数：给定 x，计算 CVaR
    # 3. 优化
    
    pass

# 测试你的实现
# allocation_cvar, cvar_value = optimize_cvar(channels, total_budget, param_uncertainty)

## 练习 2: 动态预算分配

假设预算分 4 个季度发放，每个季度可以根据前期效果调整分配。请设计一个**动态优化**策略：

- Q1: 初始分配（基于历史数据）
- Q2-Q4: 根据实际观测到的响应曲线参数，更新分配

这涉及**贝叶斯更新** + **滚动优化**。

In [ ]:
# TODO: 练习 2 - 动态预算分配

def dynamic_budget_allocation(channels_params_prior, total_budget_per_quarter, n_quarters=4):
    """
    动态预算分配（多期优化）
    
    参数:
        channels_params_prior: dict, 先验参数分布
        total_budget_per_quarter: float, 每季度预算
        n_quarters: int, 季度数
    
    返回:
        history: list, 每个季度的分配方案和效果
    """
    # TODO: 实现动态优化
    # 伪代码:
    # for q in 1 to n_quarters:
    #     1. 基于当前参数估计，优化分配
    #     2. 执行分配，观测实际效果
    #     3. 贝叶斯更新参数估计
    #     4. 进入下一季度
    
    pass

## 思考题

### 思考 1: 为什么不能只看平均 ROI？

假设：
- 渠道 A：投入 100 万，回报 300 万，ROI = 3.0
- 渠道 B：投入 10 万，回报 50 万，ROI = 5.0

如果只看 ROI，应该全投 B。但为什么实际中我们还要投 A？

**提示**：考虑边际收益递减 + 规模效应。

---

### 思考 2: 如何处理时滞效应？

某些渠道的效果有延迟：
- 品牌广告：投放后 3 个月才看到转化提升
- KOL 种草：1-2 周后才有搜索量增加

如何在优化模型中考虑时滞？

**提示**：分布式滞后模型（Distributed Lag Model）。

---

### 思考 3: 竞争对手的影响

如果竞争对手也在同一渠道投放，你的效果会受影响（拥挤效应）。如何建模？

**提示**：博弈论 + 纳什均衡。

---

### 思考 4: 在线学习与实时调整

能否设计一个**在线算法**，每天根据实时效果调整预算分配（类似 Multi-Armed Bandit）？

**提示**：Thompson Sampling / UCB 算法。

---

# 总结

## 核心要点回顾

| 概念 | 关键公式 | 实践意义 |
|------|----------|----------|
| **边际收益递减** | $R''(x) < 0$ | 不能只投一个渠道 |
| **边际 ROI 相等** | $R_1'(x_1) = R_2'(x_2) = \lambda$ | 最优分配条件 |
| **影子价格** | $\lambda = \frac{\partial R}{\partial B}$ | 预算价值 |
| **稳健优化** | $\max \mathbb{E}[R]$ 或 $\max \text{CVaR}[R]$ | 应对不确定性 |
| **交互效应** | $R(x_1, x_2) \neq R_1(x_1) + R_2(x_2)$ | 渠道不独立 |

## 实战清单

- [ ] 收集历史投放数据（至少 3 个月）
- [ ] 拟合响应曲线（Hill 方程 / Logistic）
- [ ] 定义业务约束（与团队讨论）
- [ ] 运行优化（本 Notebook 的代码）
- [ ] 敏感性分析（参数 ±20%）
- [ ] 业务评审（向老板汇报）
- [ ] 执行监控（实际 vs 预测）
- [ ] 定期更新（每季度重新优化）

## 进阶学习资源

### 论文
1. **Marketing Mix Modeling**: "Bayesian Methods for Media Mix Modeling" (Google)
2. **Robust Optimization**: "Robust Optimization for Budget Allocation" (Operations Research)
3. **Multi-Armed Bandit**: "Thompson Sampling for Budget Allocation" (NIPS)

### 工具
- **Google Lightweight MMM**: 开源营销组合建模工具
- **Facebook Robyn**: 自动化 MMM 框架
- **PyOMO**: Python 优化建模语言

### 书籍
- *"Marketing Engineering"* by Lilien & Rangaswamy
- *"Convex Optimization"* by Boyd & Vandenberghe（第4章：优化应用）

---

**恭喜你完成本 Notebook！**

你已经掌握了预算分配优化的核心方法，可以应用到实际业务中。记住：

> *"优化不是一次性的，而是持续学习和调整的过程。"*

现在，去优化你的预算吧！